In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from PIL import Image

mainDIR = os.listdir('../input/chest-xray-pneumonia/chest_xray')
print(mainDIR)

In [78]:
# To check the number of pneumonia images in train
basedir = '../input/chest-xray-pneumonia/chest_xray/'

In [79]:
train_folder = basedir + 'train/'
val_folder   = basedir + 'val/'
test_folder  = basedir + 'test/'

print(train_folder)
print(val_folder)
print(test_folder)

In [80]:
train = "../input/chest-xray-pneumonia/chest_xray/train/"

In [81]:
train_n = train_folder+'NORMAL/'
train_p = train_folder+'PNEUMONIA/'

print(len(os.listdir(train_p)))

In [104]:
num_pneumonia = len(os.listdir(os.path.join(train_folder, 'PNEUMONIA')))
num_normal = len(os.listdir(os.path.join(train_folder, 'NORMAL')))
weight_for_0 = num_pneumonia / (num_normal + num_pneumonia)
weight_for_1 = num_normal / (num_normal + num_pneumonia)

class_weight = {0: weight_for_0, 1: weight_for_1}

print(f"Weight for class 0: {weight_for_0:.2f}")
print(f"Weight for class 1: {weight_for_1:.2f}")

In [82]:
print(len(os.listdir(train_n)))

In [83]:
norm_pic = train_n + os.listdir(train_n)[500]  
pneu_pic = train_p + os.listdir(train_p)[500]

norm_disp = Image.open(norm_pic)
pneu_disp = Image.open(pneu_pic)

f = plt.figure(figsize= (15,10))
a1 = f.add_subplot(1,2,1)
img_plot = plt.imshow(norm_disp)
a1.set_title('Normal')

a2 = f.add_subplot(1, 2, 2)
img_plot = plt.imshow(pneu_disp)
a2.set_title('Pneumonia')

In [108]:
# 
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
#
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "sigmoid"))

In [85]:
# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [86]:
# Compile the model
model.compile(optimizer = 'adam' , loss = "binary_crossentropy", metrics= ['accuracy'])

In [87]:
# data augmentation
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

#train_datagen.fit(train)

In [88]:
test_datagen = ImageDataGenerator(rescale= 1./255) 

In [89]:
type(train_datagen)

In [90]:
type(test_datagen)

In [91]:
cnn.summary()

In [92]:
cnn.dtype

In [93]:
type(cnn)

In [94]:
train_set = train_datagen.flow_from_directory(train_folder, target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_set = test_datagen.flow_from_directory(val_folder,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

test_set = test_datagen.flow_from_directory(test_folder,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

In [95]:
cnn_model = cnn.fit_generator(train_set, steps_per_epoch= len(train_set) , epochs = 3, 
                              validation_data=validation_set, validation_steps= len(test_set) )

### * Epoch ilerledikçe doğruluk oranı artıyor.

In [97]:
Y_pred = cnn.predict_generator(test_set, 100)
y_pred = np.argmax(Y_pred, axis=1)

In [112]:
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           mode='min',
                           min_delta=0.0001,
                           restore_best_weights=True)

In [ ]:
history = cnn.fit(train_set,
                        steps_per_epoch=len(train_set),
                        validation_data=validation_set,
                        validation_steps=len(validation_set),
                        class_weight = class_weight,
                        epochs=3,
                        callbacks=[early_stop])

In [120]:
## plot loss for train and Validation
loss_train = cnn_model.history['loss']
loss_val = cnn_model.history['val_loss']
plt.figure(figsize=(6,4), dpi=100)
plt.plot(loss_train, 'g', label = 'Training loss')
plt.plot(loss_val, 'y', label = 'Validation loss')
plt.title("Training and Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## plot loss for train and Validation
acc_train = cnn_model.history['accuracy']
acc_val = cnn_model.history['val_accuracy']
plt.figure(figsize=(6,4), dpi=100)
plt.plot( acc_train, 'g', label = 'Training accuracy')
plt.plot( acc_val, 'y', label = 'Validation accuracy')
plt.title("Training and Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()